In [1]:
import pickle,re
import pandas as pd
import numpy as np

In [2]:
repFiles="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/IMM21/"
# rulesFile="IMM21-02-X-Regles.pkl"
# resultFile="IMM21-02-X-paradigmes-Swim1.csv"
rulesFile="IMM21-02-X-Morphomes-Regles.pkl"
resultFile="IMM21-02-X-Morphomes-Swim2.csv"
fRules=repFiles+rulesFile
percentThreshold=.1
debug=False

In [3]:
if "Morphomes" in rulesFile:
    bMorphomes=True
    tirageFile=rulesFile.replace("-Regles.pkl",".pkl")
    fTirage=repFiles+tirageFile
    with open(fTirage, 'rb') as input:
        dfTirages = pickle.load(input)
    morphomeCases=dfTirages[["case","morphome"]].drop_duplicates().to_dict()
    morphomeCases
    casesMC=morphomeCases["case"]
    morphomesMC=morphomeCases["morphome"]
    dictMorphomeCases={}
    for element in casesMC:
        dictMorphomeCases[casesMC[element]]=morphomesMC[element].split("/")
else:
    bMorphomes=False


In [4]:
# dictMorphomeCases

#### paireClasses

In [5]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            if debug: print ("le numéro de forme n'est pas dans [1,2]",n)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

#### classesPaire

In [6]:
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
        self.classeCF={}
        self.nbClasseCF={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeFormeCF=[]
        regleFormeCF=""
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            filterF1=".*"+patron.split("-")[0]+"$"
            if re.match(filterF1,forme1):
                classeFormeCF.append(patron)
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleFormeCF=patron
            filterF1=self.patrons[patron]
            if re.match(filterF1,forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseFormeCF=", ".join(classeFormeCF)
        if not idClasseFormeCF in self.classeCF:
            self.classeCF[idClasseFormeCF]={}
            self.nbClasseCF[idClasseFormeCF]=0
        if not regleFormeCF in self.classeCF[idClasseFormeCF]:
            self.classeCF[idClasseFormeCF][regleFormeCF]=0
        self.nbClasseCF[idClasseFormeCF]+=1
        self.classeCF[idClasseFormeCF][regleFormeCF]+=1
        
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=True):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            # print filterF1,forme
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme)
                    print ("pas de classe",idClasseForme)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)))
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            if debug:
                print (forme) 
                print ("pas de patron")
        return sortieForme
        

## Traitement

In [7]:
dfSwim=pd.read_csv(repFiles+resultFile,sep=";",encoding="utf8")
dfStemSpaces=dfSwim.copy()
nbLexemes=dfSwim.lexeme.count()
threshold=percentThreshold*nbLexemes
threshold


636.8000000000001

In [8]:
with open(fRules, 'rb') as input:
    regles = pickle.load(input)

In [9]:
zeroEntropy=[]
for paire in regles:
    if paire[0]!=paire[1]:
        pClasses=regles[paire].classe
        bZero=True
        nbExemples=0
        for regle in pClasses:
            if len(pClasses[regle])>1:
                bZero=False
                break
            else:
                for classe in pClasses[regle]:
                    nbExemples+=pClasses[regle][classe]
        if bZero and nbExemples>threshold:
            zeroEntropy.append((paire,nbExemples))

In [10]:
sorted(zeroEntropy, key=lambda tup: tup[0][0])

[((u'fi2P', u'pc3S'), 1011),
 ((u'fi2P', u'fi3S'), 770),
 ((u'fi2P', u'fi3P'), 760),
 ((u'fi3P', u'pc3S'), 1562),
 ((u'fi3P', u'fi2P'), 760),
 ((u'fi3P', u'fi3S'), 1159),
 ((u'fi3S', u'fi2P'), 770),
 ((u'fi3S', u'pc3S'), 1720),
 ((u'fi3S', u'fi3P'), 1159),
 ((u'ii3S', u'pi2P'), 1553),
 ((u'ii3S', u'pI2P'), 1151),
 ((u'ii3S', u'pI1P'), 988),
 ((u'ii3S', u'ii1P'), 708),
 ((u'ii3S', u'pP'), 1092),
 ((u'ii3S', u'pi1P'), 1078),
 ((u'ii3S', u'ii2P'), 1108),
 ((u'pI1P', u'ii3S'), 988),
 ((u'pI2P', u'pi2P'), 774),
 ((u'pI2P', u'ii3S'), 1151),
 ((u'pP', u'pi2P'), 685),
 ((u'pc2P', u'pc3S'), 653),
 ((u'pc3S', u'fi3P'), 1562),
 ((u'pc3S', u'pc2P'), 653),
 ((u'pc3S', u'fi3S'), 1720),
 ((u'pc3S', u'fi2P'), 1011),
 ((u'pi1P', u'ii3S'), 1078),
 ((u'pi2P', u'pP'), 685),
 ((u'pi2P', u'ii3S'), 1553),
 ((u'pi2P', u'pI2P'), 774),
 ((u'ppFS', u'pI2P'), 739),
 ((u'ppFS', u'ppMS'), 1242),
 ((u'ppFS', u'pP'), 728),
 ((u'ppFS', u'ppMP'), 962),
 ((u'ppFS', u'pi2P'), 791),
 ((u'ppMP', u'ppMS'), 1412),
 ((u'ppMP'

In [11]:
def stemSpaceForm(rules,row,contextFree=False):
    swimSortir=rules.sortirForme(row,contextFree)
    if swimSortir:
        result=swimSortir.keys()[0]
    else:
        result=np.nan
    return result

In [12]:
dfStemSpaces.loc[(dfStemSpaces["ii3P"].notnull()) & (dfStemSpaces["pi2P"].isnull()),"ii3P"].count()

156

In [13]:
nbStemSpaceForms=0
for (c1,c2),_ in zeroEntropy:
    print c1,c2
    nbStemSpaceForms+=dfStemSpaces.loc[(dfStemSpaces[c1].notnull()) & (dfStemSpaces[c2].isnull()),c1].count()
    dfStemSpaces.loc[(dfStemSpaces[c1].notnull()) & (dfStemSpaces[c2].isnull()),c2]=dfStemSpaces.loc[(dfStemSpaces[c1].notnull()) & (dfStemSpaces[c2].isnull()),c1].apply(lambda x: stemSpaceForm(regles[c1,c2],x))
nbStemSpaceForms

pI1P ii3S
fi3S fi2P
pc3S fi3P
fi3S pc3S
ii3S pi2P
pi2P pP
ii3S pI2P
fi3P pc3S
fi2P pc3S
ii3S pI1P
pI2P pi2P
ii3S ii1P
pI2P ii3S
pc3S pc2P
pi2P ii3S
pc2P pc3S
fi2P fi3S
ppFS pI2P
pi2P pI2P
ppMP ppMS
ppMS pP
fi3S fi3P
pc3S fi3S
ppMS ppMP
ppFS ppMS
ppFS pP
pc3S fi2P
ii3S pP
ppFS ppMP
fi2P fi3P
pi1P ii3S
ii3S pi1P
ppFS pi2P
ppMP ii3S
fi3P fi2P
fi3P fi3S
ii3S ii2P
pP pi2P


6409

In [14]:
if bMorphomes:
    for c1 in dictMorphomeCases:
        for c2 in dictMorphomeCases[c1]:
            if c1!=c2:
                print c1,"=>",c2
                dfStemSpaces[c2]=dfStemSpaces[c1]
    

ppMS => ppMP
is3P => is2S
is3P => is1S
is3S => ai3S
is3S => ai2S
ii3S => ii1S
ii3S => ii2S
ii3S => ii3P
ppFS => ppFP
ps3P => ps2S
ps3P => ps3S
ps3P => pi3P
ps3P => ps1S
pi3S => pi1S
pi3S => pI2S
pi3S => pi2S
pc3S => pc2S
pc3S => pc3P
pc3S => pc1S
pc3S => fi1S
fi3S => fi2S
fi3P => fi1P


In [15]:
dfStemSpaces.to_csv(repFiles+resultFile.replace(".csv","-StemSpace.csv"),index=None,sep=";",encoding="utf8")